In [ ]:
%matplotlib inline
import sys, os
shared_dir = '/nsls2/software/mx/lix/pylibs'
sys.path = ['.', f'{shared_dir}/py4xs', f'{shared_dir}/lixtools']+sys.path
from lixtools.notebooks import display_solHT_data, display_HPLC_data
from lixtools.hdf import h5sol_ref,h5sol_HT, h5sol_HPLC
import lixtools.notebooks 
from lixtools.atsas import gen_report,gen_atsas_report
#from py4xs.hdf import h5exp,h5xs,lsh5, h5py, trans_mode
import warnings
warnings.filterwarnings("ignore")
import requests
from io import StringIO
import numpy as np
import matplotlib.pylab as plt2
import matplotlib.pyplot as plt
from lixtools.mailin import checkHolderSpreadsheet
from py4xs.slnxs import filter_by_similarity
from py4xs.hdf import h5exp
import pandas as pd
import requests
from io import StringIO
global proc_path
#import pytables
atsas_path="/nsls2/data/lix/shared/software/ATSAS-3.0.3-1/bin/"
fn = "00template00.h5"
exp_fn = "00exp00.h5"

def get_UV_data(sample_name):
    """This will fetch the 280 UV data from the working directory and is used by h5_attach_hplc"""
    uv_file = f"{sample_name}.dx_DAD1E.CSV"
    df = pd.read_csv(f'{sample_name}/{uv_file}')
    k=df.to_numpy(dtype=float) 
    return k

def h5_attach_hplc(fn_h5, grp_name=None):
    """ the hdf5 is assumed to contain a structure like this:
        LIX_104
        == hplc
        ==== data
        == primary (em, scattering patterns, ...)
        
        attach the HPLC data to the specified group
        if the group name is not give, attach to the first group in the h5 file
    """
    f = h5py.File(fn_h5, "r+")
    if grp_name == None:
        grp_name = list(f.keys())[0]
    sample_name=list(f.keys())[0]
    k=get_UV_data(sample_name)

    
    # this group is created by suitcase if using flyer-based hplc_scan
    # otherwise it has to be created first
    # it is also possible that there was a previous attempt to populate the data

    if 'hplc' in f[f"{grp_name}"].keys():
        grp = f["%s/hplc/data" % grp_name]
    else:
        grp = f.create_group(f"{grp_name}/hplc/data")
        
    key_list=list(grp.keys())
    for g in grp:
        if g in key_list:
            print("warning: %s already exists, deleting ..." % g)
            del grp[g]
    else:
        print("no UV_data previously present")
    d=np.asarray(k)
    #print(d)
    dset=grp.create_dataset('[LC Chromatogram(Detector A-Ch1)]', data=d)
    dset[:]=d
    f.close()

In [ ]:
##SEC report ##need a function to start report generation
from IPython.display import display, HTML
import warnings
import os

In [1]:
def create_SEC_report(fn, exp_fn, q_ranges=None, plot_peaks=True):
    print(fn)
    
    fig, axes = plt.subplots(1,1, figsize=(10,4))
    sn = os.path.basename(fn).split(".")
    if sn[-1]!="h5":
        print(f"It appears that {fn} is not a valid h5 file. Did you include the whole .h5 filename?")
    else:
        sn=sn[0]
    dexp=h5exp(exp_fn)
    dt=h5sol_HPLC(f'{sn}.h5', [dexp.detectors, dexp.qgrid])
    dt.load_data()
    dt.load_d1s()
    d_i=create_scalar(dt, sn=sn, q_ranges=q_ranges)
    x, peaks, props, p = find_SEC_peaks(d_i)
    w,left,right,peak_mask = SEC_peak_width(x, p)
    if plot_peaks:
        axes.set_title("Peak finding")
        axes.axvspan(left,right,alpha=0.2,label="peak")
        axes.plot(peak_mask*int(np.max(x)))
        axes.plot(d_i[0])
        axes.plot(x,'.')
    s,b = SEC_auto_process(dt,sn,left,right)

In [ ]:
def create_scalar(dt,sn, q_ranges= None):
    if q_ranges is None:
        q_ranges = [[0.02, 0.05]]
    idx = [(dt.qgrid>i_minq) & (dt.qgrid<i_maxq) for [i_minq,i_maxq] in q_ranges]
    dd= dt.d1s[sn]["merged"]
    nd=len(dd)
    nq=len(idx)
    d_i = np.zeros((nq,nd))
    for i in range (len(dd)):
        for j in range(nq):
            d_i[j][i] = dd[i].data[idx[j]].sum(axis=0)
    return d_i
    

In [ ]:
###find peaks on scalar data--prominence needs to be tested on overlapping peaks.  For report, just get an idea of peaks and then go back and do SVD, EFA, etc
###returns index
from scipy.signal import find_peaks, peak_widths
def find_SEC_peaks(d_i):
    x=d_i[0] - np.min(d_i[0])
    peaks, props = find_peaks(x, prominence=0.05*np.max(x))
    p=peaks[np.argmax(props["prominences"])]##highest index
    return x, peaks, props, p

In [ ]:
##find peaks and define left right line at rel_height.  Returns boolean of zeros to define peak
def SEC_peak_width(x, p, rel_height = 0.5):
    w=peak_widths(x, [p], rel_height=rel_height)
    left = int(np.floor(w[2][0]))
    right = int(np.ceil(w[3][0]))
    peak_mask = np.zeros_like(x,dtype=bool)
    peak_mask[left:right+1]=True
    return w, left, right, peak_mask

In [ ]:
def SEC_auto_process(dt, sn, left, right, buffer_frame_range = None):
    '''
    ###final plots should include peaks, found peaks, estimated buffer range, peak found frame range, average plots and subtracted plots, charomatograph
    ### take frames 0-40 and use as buffer subtraction for apex of peak. Using subtract_buffer function.  Need to define regions  Then print a report 
    ##define buffer region: make sure it is not in d_i bool TRUE
    '''
    left=int(left)
    right=int(right)
    sample_frame_range = [left+1,right+1]
    if buffer_frame_range == None:
        buffer_frame_range = ('0-20')
    else:
        buffer_frame_range=buffer_frame_range
    if type(buffer_frame_range) is str:
        f1,f2 = buffer_frame_range.split('-')
        buffer_frame_range = range(int(f1), int(f2))
    display(HTML(f'Buffer frames are {buffer_frame_range}'))
    sample_frame_range = range(int(left+1), int(right+1))
    display(HTML(f'Using sample frame range {sample_frame_range}'))
    for i in buffer_frame_range:
        if i in sample_frame_range:
            raise Exception("Buffer frame range is within peak!")
    ##plot average sample data
    sample_lists  = [dt.d1s[sn]['merged'][i] for i in sample_frame_range]
    qmax_for_weight = 0.3
    sample_lists[0].avg(sample_lists[1:], plot_data=True, weighted=True, debug=False)
    ##plot average buffer data
    buffer_lists=[dt.d1s[sn]['merged'][i] for i in buffer_frame_range]

    #display(HTML(f'Buffer frames are {buffer_frame_range} after buffer_lists generated'))
    buffer_lists[0].avg(buffer_lists[1:], plot_data=True, weighted=True, debug=False)
    
    
    ##plot subtracted data
    dt.subtract_buffer(sample_frame_range=sample_frame_range,buffer_frame_range=buffer_frame_range, sc_factor=0.95)
    ##plot chromatograph
    return sample_frame_range, buffer_frame_range

In [ ]:
create_SEC_report(fn, exp_fn)